# Introduction to GitHub Repo and Gists Reporting

<p>This Notebook provides a basic reporting tool that creates a report based on either a users or an organizations repositories.  </p>
<p>For organizations with a large number of repositories, a Filter List can be used to limit the number of repos returned.</p>
<p>This Notebook also provides a basic reporting tool that creates a report of a users' Gists.  </p>

## About this Notebook
This Notebook uses the following:

* Python 3 
* toc2 nbextension (optional) for displaying Table of Contents.
* MikTex for nbconversion (optional) for converting notebook to pdf

## References and Info
See command below for converting Notebook to web pdf without code cells and prompts
```
jupyter nbconvert .\GitHubReports.ipynb --to webpdf --no-input --output myReport.pdf
```

# GitHub Report
<p>This report is a basic report based on either a users or an organizations GitHub repositories.  </p>

## Prerequisites
<p> This report requires a separate Git credentials file to be placed in same directory as notebook and a reporting strategy (choose eiter user or org)
    </p>

### GitHub Credentials File Setup

<div class="alert alert-block alert-info">
<b>Create security file:</b> 
<p>In the same directory as this Notebook, create a file - gitsecrets.py - and add the following:<br>
    GITHUB_USER= Your Git User Name  <br>
    GITHUB_TOKEN= Your Git Token
</p>
</div>

<div class="alert alert-block alert-warning"><b>GitHub API Limits: </b>You may reach a limit of API calls that GitHub will allow, ex. 2000 calls/minute.  Try to limit the number of repos/folders/files. </div>


## Initialize Report
<p>Required steps before report can be executed</p>

### Import packages and Set Authorization
Credentials file - ***gitsecrets.py*** - needs to be available

In [ ]:
import requests
import pprint
from gitsecrets import GITHUB_USER,GITHUB_TOKEN
from IPython.display import display, Math, HTML, Markdown

display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 70em; }</style>"))

# Create auth for Git credentials
auth=requests.auth.HTTPBasicAuth( GITHUB_USER  ,GITHUB_TOKEN )
print('Authorization set up for user: ', GITHUB_USER)

### Fetch Repos
You will be prompted for User or Org

In [ ]:
print('Will the report be for a User or Org (User/Org)?')
type = input()
org = ''
if type == "U" :
    print('Your report will be base on your Git User')
    repoUrl = 'https://api.github.com/user/repos' 
    repoResponse = requests.get(repoUrl, auth=auth)
    print('Authorization Confirmed for user: ' + GITHUB_USER + ' Repo Url is: '+ repoUrl)
elif type == "O":
    print('Report will be based on your organization, please type name of org:')
    org = input()
    repoUrl = 'https://api.github.com/orgs/' + org + '/repos'
    repoResponse = requests.get(repoUrl, auth=auth)
    print('Authorization Confirmed for user: ' + GITHUB_USER + ' Repo Url is: '+ repoUrl)
else :
    print ('Error, try again')
reposFilterUrls =[]

## Repos/Filter List 
* Filter List (optional) 
* Repos List (required)

### Create Repo Filter List (optional)
<p>Append name of desired repos to the reposFilterUrls List, as required</p>
Example:  reposFilterUrls.append('myTestRepo')

In [ ]:
tmpReposFilterUrls = []
# Add Filters as required as show below
# tmpReposFilterUrls.append('JupyterCloud')
tmpReposFilterUrls.append('truccr_oci-iac')
# tmpReposFilterUrls.append('truccr_oci-ca-dumptruck')
# tmpReposFilterUrls.append('truccr_oci-manual-transmission')
# tmpReposFilterUrls.append('truccr_atlas')

print( 'Number of Filters: '  ,len(tmpReposFilterUrls))
print('Filter List:')
reposFilterUrls = []
for rFilter in tmpReposFilterUrls:
    if type == 'U':
        urlExtn = 'user/repos/'
    else:
        urlExtn = 'repos/' + org + '/'
    newFilter = 'https://api.github.com/' + urlExtn + rFilter 
    reposFilterUrls.append(newFilter  )
    print(newFilter)


### Create Repo List (required)
<p>Run cell and verify repo count (Note: max repos is 30)</p>

In [ ]:
repos =[]
if reposFilterUrls is not None and len(reposFilterUrls) > 0:
    for reposFilterUrl in reposFilterUrls:
        resp = repoResponse = requests.get(reposFilterUrl, auth=auth)
        repos.append(resp.json())
else:
    for repoR in repoResponse.json():
        repos.append(repoR)
print( 'Number of Repos Found: '  ,len(repos))


## Repository Report

In [ ]:
for repo in repos :
    print( 'Repo:',repo["name"])
    print('    Private:     ', repo['private'])
    print('    HTTPS URL:   ', repo['clone_url'])
    print('    SSH URL:     ', repo['ssh_url'])    
    contentsUrl = repo['contents_url']
    print('    Contents URL:' ,contentsUrl.replace('{+path}',''))
    contents = requests.get(contentsUrl.replace('{+path}',''),auth=auth)
    cFileList = []  ;   cDirList = []
    [cFileList.append(x) for x in contents.json() if x['type'] == 'file' ]
#- Files
    print('    Files:')
    for content in cFileList:
        print('       ', content['name'])
    [cDirList.append(x) for x in contents.json() if x['type'] == 'dir' ]
#- Folders
    print('    Folders:')
    for content in cDirList:
        print('       ', '/' + content['name'])

#- Branches
    print('    Branches:')
    branchs = requests.get(repo['branches_url'].replace('{/branch}',''),auth=auth)
    for branch in branchs.json():
        print('         ' ,branch['name'])
        branchContents = requests.get(repo['branches_url'].replace('{/branch}','/'+branch['name'])
                                      ,auth=auth)
        branchContent = branchContents.json()['commit']['commit']
        print('              Author:',branchContent['author'] ['name'])
        print('              Date:  ' ,branchContent ['author'] ['date'] )             
#-Folder Contents
    print('    Folder Contents:')
    for folder in cDirList:
        print('        /'+ folder['name'])
        fContents = requests.get(folder['url'] , auth=auth   )
        cFileList2 = []  ;   cDirList2 = []
        [cFileList2.append(x) for x in fContents.json() if x['type'] == 'file' ]
        for content in cFileList2:
            print('           '+ content['name'])
        [cDirList2.append(x) for x in fContents.json() if x['type'] == 'dir' ]            
        for content in cDirList2:
            print('           /'+ content['name'])     
            folder2 = folder['name'] + '/' + content['name']
#             print(contentsUrl.replace('{+path}',folder2 ))
            fContents2 = requests.get(contentsUrl.replace('{+path}',folder2),auth=auth)
            cFileList3 = []  ;   cDirList3 = []
            [cFileList3.append(x) for x in fContents2.json() if x['type'] == 'file' ]    
            for content2 in cFileList3:
                print('               ', content2['name'] )
            [cDirList3.append(x) for x in fContents2.json() if x['type'] == 'dir' ] 
            for content2 in cDirList3:
                print('               /'+ content2['name'] )      
                newDirName =  folder['name']  +'/'+content['name'] + '/' + content2['name']
                fContents3 = requests.get(contentsUrl.replace('{+path}',newDirName),auth=auth)
                cFileList4 = []  ;   cDirList4 = []
                [cFileList4.append(x) for x in fContents3.json() if x['type'] == 'file' ]    
                for content3 in cFileList4:
                    print('                   ', content3['name'] )
                [cDirList4.append(x) for x in fContents3.json() if x['type'] == 'dir' ]
                for content3 in cDirList4:
                    print('                   /'+ content3['name'] )   
                    newDirName2 =  folder['name']  +'/'+content['name'] + '/' + content2['name'] \
                                 + '/' + content3['name']
                    fContents4 = requests.get(contentsUrl.replace('{+path}',newDirName2),auth=auth)
                    cFileList5 = []  ;   cDirList5 = []
                    [cFileList5.append(x) for x in fContents4.json() if x['type'] == 'file' ]    
                    for content4 in cFileList5:
                        print('                       ', content4['name'] )
                    [cDirList5.append(x) for x in fContents4.json() if x['type'] == 'dir' ]
                    for content4 in cDirList5:
                        print('                   /'+ content4['name'] )   
#- Issues
    print('    Open Issues:')
    issues = requests.get(repo['issues_url'].replace('{/number}',''),auth=auth)
    try:
        for issue in issues.json():
            if 'open' in issue['state']:
                print('         ' ,str(issue['number']) + ' - ' +  issue['body'])   
    except:
        pass
    print('\n---' )
print('----------')


In [ ]:
for branch in branchs.json():
    print('         ' ,branch['name'])
    if branch['name'] == 'upd-terraform-resource-names':
        myBranch = branch
        myUrl = branch ['commit'] ['url']
        break
# pprint.pprint(myBranch)
myBranchContents = requests.get(myUrl,auth=auth)
# pprint.pprint(myBranchContents.json()['commit']['tree'] ['url']   )   
myBranchTreeUrl = myBranchContents.json()['commit']['tree'] ['url'] 
print('My Branch URL: ' , myBranchTreeUrl )

In [ ]:
x =requests.get(myBranchTreeUrl,auth=auth)

for tree in x.json()['tree']  :
    if tree['type'] == 'tree':
        print(tree['path'] + '/'       )
        x2 = requests.get(tree['url'] , auth=auth)
        for tree2 in x2.json()['tree']:
            print( '    ',  tree2['path'] + (  '/'  if tree2['type'] == 'tree' else ''  ) )
            if tree2['type'] == 'tree' :
                x3 = requests.get(tree2['url'] , auth=auth)
                for tree3 in x3.json()['tree']:
                    print( '        ',  tree3['path'] + (  '/'  if tree3['type'] == 'tree' else ''  ) )
                    if tree3['type'] == 'tree' :
                        x4 = requests.get(tree3['url'] , auth=auth)
                        for tree4 in x4.json()['tree']:
                            print( '            ',  tree4['path'] + (  '/'  if tree4['type'] == 'tree' else ''  ) )
                            if tree4['type'] == 'tree' :
                                x5 = requests.get(tree4['url'] , auth=auth)
                                for tree5 in x5.json()['tree']:
                                    print( '                ',  tree5['path'] + (  '/'  if tree5['type'] == 'tree' else ''  ) )


In [ ]:

commentsUrl =  requests.get(  myBranchContents.json()['comments_url']  , auth=auth)
pprint.pprint(commentsUrl.json())

# Gists Report

## Import Packages and Fetch Gists

In [ ]:
import requests
import pprint
from gitsecrets import GITHUB_USER,GITHUB_TOKEN
# Create auth for Git credentials
auth=requests.auth.HTTPBasicAuth( GITHUB_USER  ,GITHUB_TOKEN )

userRepoUrl = 'https://api.github.com/gists'
myGistsUrl = userRepoUrl
repoResponse = requests.get(myGistsUrl, auth=auth)

## Gists Report

In [ ]:
gists = repoResponse.json()
for gist in gists:
    print( gist['description'])
    print('    Updated: ', gist['updated_at']  )
    print('    Type:    ', 'Public' if gist['public'] else 'Private' )
    print('    Html Url: ', gist['html_url']  )
    print('    Comments: ', gist['comments']  )
    
    forks = requests.get(gist['forks_url'],auth=auth)
    print('    Forks:    ',str(len(forks.json())))
    files = gist['files']
    print('    Files:')
    for file,extra in files.items():
        print('        File: ',file)
    print('--')
print('\n----------')